In [1]:
import os
import pandas as pd
import re

# Ruta de la carpeta de datos
data_folder = 'data'

# Lista para almacenar los DataFrames
dfs = []

# Iterar sobre los archivos en la carpeta
for filename in os.listdir(data_folder):
    if filename.endswith('.xlsx') and 'robust_cash_flow' in filename:
        file_path = os.path.join(data_folder, filename)
        df = pd.read_excel(file_path)
        #df['nombre_archivo'] = filename  # Agrega columna con el nombre del archivo
        # Extrae año (20xx) y Q (q1, q2, q3, q4) del nombre del archivo
        match = re.search(r'(20\d{2})', filename)
        year = match.group(1) if match else ''
        match_q = re.search(r'(q[1-4])', filename, re.IGNORECASE)
        q = match_q.group(1).upper() if match_q else ''
        df['anio_q'] = year +" "+ q
        dfs.append(df)

# dfs contiene todos los DataFrames importados, cada uno con las columnas 'nombre_archivo', 'anio' y 'trimestre'

In [2]:
len_dfs = len(dfs)
print(f"Se han importado {len_dfs} archivos de flujo de caja robusto.")

Se han importado 7 archivos de flujo de caja robusto.


In [3]:
dfs[0]

,LTM Q3 2023,Q4 2022,Q3 2023,Q2 2023,nombre de la métrica,Q1 2023,anio_q
0,$312.9,$132.1,$44.2,$89.6,Net cash generated from operating activities (M$),$47.0,2023 Q3
1,($120.6),$40.5,($69.7),($37.4),Free cash flow (M$)1,($54.0),2023 Q3
2,$304.9,$62.6,$98.7,$77.2,Media Luna Project capex (M$)1,$66.4,2023 Q3
3,$184.3,$103.1,$29.0,$39.8,Free cash flow prior to Media Luna Project (M$)1,$12.4,2023 Q3
4,427.9,121.9,81.8,105.7,Gold sold (koz),118.5,2023 Q3
5,$816,$711,"$1,086",$848,Total cash costs ($/oz)1,$709,2023 Q3
6,"$1,194","$1,034","$1,450","$1,308",All-in sustaining costs ($/oz)1,"$1,079",2023 Q3
7,"$1,890","$1,784","$1,944","$1,960",Average realized gold price ($/oz)1,"$1,899",2023 Q3


In [4]:
dfs1 = []
import re

palabras_clave = ["Métrica", "nombre", "Key Metric","nombre de la métrica","Metric"]

for i, df in enumerate(dfs):
   # print(df.columns)
   # print(df.head())
    
    valor_anio_q = str(df["anio_q"].iloc[0]).strip()
    # Extraer año y Q
    match = re.search(r'(\d{4})\s*[-_ ]?\s*(Q\d)', valor_anio_q, re.IGNORECASE)
    if match:
        anio = match.group(1)
        q = match.group(2).upper()
    else:
        print(f"No se pudo extraer año y Q de '{valor_anio_q}'")
        continue
    
    # Buscar columnas que contengan tanto el año como el Q
    cols_q_anio = [col for col in df.columns if anio in col and q in col.upper()]
    
    # Columnas de métricas
    cols_metricas = [col for col in df.columns if any(p.lower() in col.lower() for p in palabras_clave)]
    
    columnas_deseadas = cols_metricas + cols_q_anio# +["anio_q"]
    print(f"DataFrame {i+1} - Columnas seleccionadas: {columnas_deseadas}")
    df_filtrado = df[columnas_deseadas].copy()
    df_filtrado = df_filtrado.rename(columns={df_filtrado.columns[0]: "Metric"})

    print(df[columnas_deseadas].head())
    dfs1.append(df_filtrado)


DataFrame 1 - Columnas seleccionadas: ['nombre de la métrica', 'LTM Q3 2023', 'Q3 2023']
                                nombre de la métrica LTM Q3 2023  Q3 2023
0  Net cash generated from operating activities (M$)      $312.9    $44.2
1                               Free cash flow (M$)1    ($120.6)  ($69.7)
2                     Media Luna Project capex (M$)1      $304.9    $98.7
3   Free cash flow prior to Media Luna Project (M$)1      $184.3    $29.0
4                                    Gold sold (koz)       427.9     81.8
DataFrame 2 - Columnas seleccionadas: ['métrica', 'LTM Q1 2023', 'Q1 2023']
                                             métrica LTM Q1 2023  Q1 2023
0  Net cash generated from operating activities ($M)      $408.4    $47.0
1                                Free cash flow ($M)       $91.6  ($54.0)
2                      Media Luna Project capex ($M)      $191.1    $66.4
3    Free cash flow prior to Media Luna Project ($M)      $282.7    $12.4
4                    

In [5]:
for i in range(len(dfs1)):
    print(i)
    print(dfs1[i].head(2))
    print(dfs1[i]["Metric"])

0
                                              Metric LTM Q3 2023  Q3 2023
0  Net cash generated from operating activities (M$)      $312.9    $44.2
1                               Free cash flow (M$)1    ($120.6)  ($69.7)
0    Net cash generated from operating activities (M$)
1                                 Free cash flow (M$)1
2                       Media Luna Project capex (M$)1
3     Free cash flow prior to Media Luna Project (M$)1
4                                      Gold sold (koz)
5                             Total cash costs ($/oz)1
6                      All-in sustaining costs ($/oz)1
7                  Average realized gold price ($/oz)1
Name: Metric, dtype: object
1
                                              Metric LTM Q1 2023  Q1 2023
0  Net cash generated from operating activities ($M)      $408.4    $47.0
1                                Free cash flow ($M)       $91.6  ($54.0)
0    Net cash generated from operating activities ($M)
1                            

In [6]:

#Edfs_alineados = [df.set_index("Metric") for df in dfs1]
##df_concat = pd.concat(dfs_alineados, axis=1)
#df_concat.head()

# Concatenar los DataFrames de dfs1 de forma horizontal (por columnas)
df_concat = pd.concat(dfs1, axis=1)

# Mostrar las primeras filas del DataFrame resultante
df_concat.head()

,Metric,LTM Q3 2023,Q3 2023,Metric,LTM Q1 2023,Q1 2023,Metric,LTM (Q2 2024),Q2 2024,Metric,Q1 2024,LTM (Q1 2024),Metric,Q4 2023,Metric,Q2 2023,LTM Q2 2023,Metric,LTM (Q3 2024),Q3 2024
0,Net cash generated from operating activities (M$),$312.9,$44.2,Net cash generated from operating activities ($M),$408.4,$47.0,Net cash generated from operating activities (M$),$341.4,$97.4,Net cash generated from operating activities (M$),$79.8,$333.6,Net cash generated from operating activities (M$),$120.0,Net cash generated from operating activities (M$),$89.6,$371.1,Net cash generated from operating activities (M$),$446.7,$149.5
1,Free cash flow (M$)1,($120.6),($69.7),Free cash flow ($M),$91.6,($54.0),Free cash flow (M$),($205.4),($62.3),Free cash flow (M$)1,($49.1),($180.5),Free cash flow (M$)1,($24.3),Free cash flow (M$),($37.4),($18.9),Free cash flow (M$),($136.4),($0.7)
2,Media Luna Project capex (M$)1,$304.9,$98.7,Media Luna Project capex ($M),$191.1,$66.4,Media Luna Project capex (M$),$457.3,$108.2,Media Luna Project capex (M$)1,$126.4,$426.3,Media Luna Project capex (M$)1,$124.0,Media Luna Project capex (M$),$77.2,$238.7,Media Luna Project capex (M$),$472.5,$113.9
3,Free cash flow prior to Media Luna Project (M$)1,$184.3,$29.0,Free cash flow prior to Media Luna Project ($M),$282.7,$12.4,Free cash flow prior to Media Luna Project (M$),$251.9,$45.9,Free cash flow prior to Media Luna Project (M$)1,$77.3,$245.8,Free cash flow prior to Media Luna Project (M$)1,$99.7,Free cash flow prior to Media Luna Project (M$),$39.8,$219.8,Free cash flow prior to Media Luna Project (M$),$336.1,$113.2
4,Gold sold (koz),427.9,81.8,Gold sold (koz),483.6,118.5,Gold sold (koz),445.7,113.5,Gold sold (koz),111.6,437.9,Gold sold (koz),138.8,Gold sold (koz),105.7,466.0,Gold sold (koz),486.1,122.1


In [7]:
import re

# Encuentra las columnas que corresponden a trimestres (formato QX YYYY)
quarter_cols = [col for col in df_concat.columns if re.match(r'Q[1-4] \d{4}', str(col))]

# Ordena las columnas de los trimestres cronológicamente
quarter_cols_sorted = sorted(
    quarter_cols,
    key=lambda x: (int(x.split()[1]), int(x.split()[0][1]))
)

# Reordena el DataFrame: primero 'Metric', luego los trimestres ordenados, luego el resto
cols_order = (
    ['Metric'] +
    quarter_cols_sorted 
)
df_concat = df_concat[cols_order]


In [8]:
df_concat

,Metric,Metric,Metric,Metric,Metric,Metric,Metric,Q1 2023,Q2 2023,Q3 2023,Q4 2023,Q1 2024,Q2 2024,Q3 2024
0,Net cash generated from operating activities (M$),Net cash generated from operating activities ($M),Net cash generated from operating activities (M$),Net cash generated from operating activities (M$),Net cash generated from operating activities (M$),Net cash generated from operating activities (M$),Net cash generated from operating activities (M$),$47.0,$89.6,$44.2,$120.0,$79.8,$97.4,$149.5
1,Free cash flow (M$)1,Free cash flow ($M),Free cash flow (M$),Free cash flow (M$)1,Free cash flow (M$)1,Free cash flow (M$),Free cash flow (M$),($54.0),($37.4),($69.7),($24.3),($49.1),($62.3),($0.7)
2,Media Luna Project capex (M$)1,Media Luna Project capex ($M),Media Luna Project capex (M$),Media Luna Project capex (M$)1,Media Luna Project capex (M$)1,Media Luna Project capex (M$),Media Luna Project capex (M$),$66.4,$77.2,$98.7,$124.0,$126.4,$108.2,$113.9
3,Free cash flow prior to Media Luna Project (M$)1,Free cash flow prior to Media Luna Project ($M),Free cash flow prior to Media Luna Project (M$),Free cash flow prior to Media Luna Project (M$)1,Free cash flow prior to Media Luna Project (M$)1,Free cash flow prior to Media Luna Project (M$),Free cash flow prior to Media Luna Project (M$),$12.4,$39.8,$29.0,$99.7,$77.3,$45.9,$113.2
4,Gold sold (koz),Gold sold (koz),Gold sold (koz),Gold sold (koz),Gold sold (koz),Gold sold (koz),Gold sold (koz),118.5,105.7,81.8,138.8,111.6,113.5,122.1
5,Total cash costs ($/oz)1,Total cash costs ($/oz),Total cash costs ($/oz),Total cash costs ($/oz)1,Total cash costs ($/oz)1,Total cash costs ($/oz),Total cash costs ($/oz),$709,$848,"$1,086",$885,$918,"$1,014",$926
6,All-in sustaining costs ($/oz)1,All-in sustaining costs ($/oz),All-in sustaining costs ($/oz),All-in sustaining costs ($/oz)1,All-in sustaining costs ($/oz)1,All-in sustaining costs ($/oz),All-in sustaining costs ($/oz),"$1,079","$1,308","$1,450","$1,073","$1,202","$1,239","$1,101"
7,Average realized gold price ($/oz)1,Average realized gold price ($/oz),Average realized gold price ($/oz),Average realized gold price ($/oz)1,Average realized gold price ($/oz)1,Average realized gold price ($/oz),Average realized gold price ($/oz),"$1,899","$1,960","$1,944","$1,995","$2,023","$2,193","$2,313"
